In [1]:
import numpy as np
import pyvista as pv
import torch
import sys

In [ ]:
import torch
torch.cuda.is_available()

In [24]:
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int((size_bytes).bit_length() // 10)  # 根据大小选择合适的单位
    p = 1024 ** i
    size = round(size_bytes / p, 2)
    return f"{size} {size_name[i]}"

In [46]:

bottom = 900
height = 940

density = 10

sphere = np.ones((100, 100, 2000), dtype=bool)

radius = 30

center = 50


In [10]:
etching = np.load('./bosch_data_1003_ratio08/bosch_sf_step_6_sf.npy')

In [12]:
etching = np.load('./bosch_data_1002_ratio08/bosch_sf_step_6_sf.npy')

In [ ]:
etching.shape

In [43]:
etching = np.load('./bosch_data_0926_ratio08/bosch_sf_step_1_c4f8.npy')

In [38]:
etching = np.load('./bosch_data_0926_ratio08/bosch_sf_step_0_Ar.npy')

In [43]:
etching = np.load('./bosch_sf_step1.npy')

In [ ]:
etching.shape

In [34]:
from surface_normalize_sf import surface_normal

center_with_direction=np.array([[35,100,75]])
range3D=np.array([[0, 70, 0, 100, 0, 150]])
InOrOut=[1]
yield_hist=np.array([None])
celllength=1e-5
surface =  surface_normal(center_with_direction, range3D, InOrOut,celllength, yield_hist)

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))


point_cloud = pv.PolyData(planes[:, 3:])
vectors = planes[:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=100000,
    factor=5,
)

# Display the arrowscyan

sphere = pv.Sphere(radius=4, center=(21, 75, 114.5))
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.sum(etching[:, :, :, :], axis=-1)==0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

# mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
# mask = mask.indices().numpy().T

# maskmesh = pv.PolyData(mask)
# maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
# maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.sum(etching[:, :, bottom-250:bottom, :], axis=-1)==0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-250:bottom,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:, :, bottom-250:bottom,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :, bottom-250:bottom, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-250:bottom,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:, :, bottom-250:bottom,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:50, :, bottom-250:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, bottom-250:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:50, :, bottom-250:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:50, :, bottom-100:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, bottom-100:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:50, :, bottom-100:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()